In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import pyfits as pf
import shutil as sh

from ConfigParser import ConfigParser
from os import makedirs
from os.path import join, abspath, basename, exists
from argparse import ArgumentParser
from datetime import datetime
from time import sleep
from glob import glob
from copy import copy

from numpy import arange, zeros
from numpy.random import uniform
from mpi4py import MPI

from pytransit import MandelAgol as MA
from exotk.utils.orbits import as_from_rhop
from pybls import BLS

In [3]:
cp = ConfigParser()
cp.read('test.cfg');

In [4]:
set_name = cp.get('Sample Set', 'name')
sample_size = int(cp.get('Sample Set', 'size'))
random.seed(int(cp.get('Sample Set', 'random_seed')))

In [5]:
data_dir = cp.get('Sample Set', 'raw_dir')
set_dir = join('sets',set_name)
all_files = glob(join(data_dir,'kt*.fits'))
raw_files = random.permutation(all_files)[:sample_size]
set_files = [join(set_dir,basename(f)) for f in raw_files]

In [6]:
if not exists(set_dir):
    makedirs(set_dir)
for f1,f2 in zip(raw_files,set_files):
    sh.copy2(f1,f2)

In [7]:
tm =  MA(supersampling=10)

for i,f in enumerate(set_files):
    stellar_density = normal(1.7,0.1)
    period          = uniform(0.7,20)
    radius_ratio    = uniform(0.01,0.18)
    zero_epoch      = uniform(0,period)
    semi_major_axis = as_from_rhop(stellar_density,period)
    inclination     = 0.5*pi
    
    hdul = pf.open(f, mode='update')
    hdu = hdul[1]
    hdu.data['pdcsap_flux'] *= tm.evaluate(hdu.data['time'], radius_ratio, [0.4,0.1],
                                           zero_epoch, period, semi_major_axis, inclination)
    hdul[1].header['inj_rho'] = stellar_density
    hdul[1].header['inj_per'] = period
    hdul[1].header['inj_k']   = radius_ratio
    hdul[1].header['inj_t0']  = zero_epoch
    hdul[1].header['inj_sma'] = semi_major_axis
    hdul[1].header['inj_inc'] = inclination
    hdul.flush()
    hdul.close()

In [8]:
from os.path import relpath

In [ ]:
np.fromstring()